`Course Instructor`: **John Chiasson**

`Author (TA)`: **Ruthvik Vaila**

# Note
* Here we shall see the effect of iterator and generator while streaming data into a model using `.tfrecord`.
* Learn more about Python [generator](https://towardsdatascience.com/python-generators-393455aa48a3), [iterators](https://anandology.com/python-practice-book/iterators.html).
* Keep an eye on RAM usage when compared to iterator way of training a NN.
* Tested on `Python 3.7.5` with `Tensorflow 1.15.0` and `Keras 2.2.4`. 
* Tested on `Python 2.7.17` with `Tensorflow 1.15.3` and `Keras 2.2.4`.

# Imports

In [1]:
import sys
sys.version

'3.7.5 (default, Nov  7 2019, 10:50:52) \n[GCC 8.3.0]'

In [2]:
import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
from tensorflow.python import keras as keras
from tensorflow.python.client import device_lib
import numpy as np
import sys, pickle, os
import h5py, time, inspect
import IPython.display as display
import IPython
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [3]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12055725098772028220,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5635111993675007370
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4951416259726132291
 physical_device_desc: "device: XLA_GPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7400190772
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5740985474596317195
 physical_device_desc: "device: 0, name: GeForce RTX 2080 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
tf.__version__

'1.15.0'

In [5]:
keras.__version__

'2.2.4-tf'

# `Batch_Size` and `Buffer_Size` setting

In [6]:
BATCH_SIZE = 32
SHUFFLE_BUFFER = BATCH_SIZE**2
EPOCHS = 3
STEPS_PER_EPOCH = 112799 / BATCH_SIZE # number of batches per epoch.

# Parser

In [7]:
def parser(record):
    image_feature_description = {
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.VarLenFeature(tf.float32),
    }
    parsed = tf.io.parse_single_example(record, image_feature_description)
    
    image = parsed['image_raw']
    image = tf.sparse.to_dense(image,default_value = 0)
    label = tf.cast(parsed["label"], tf.int32)
    
    #return {"image_data": image}, label
    return image, label
    

# Model

In [8]:
#Combine it with keras
def smol_model(inputs=None, outputs=None):
    model_input = keras.layers.Input(tensor=inputs)

    #Build your network
    model_output = keras.layers.Dense(1500,activation='relu')(model_input)
    model_output = keras.layers.Dense(47,activation='softmax')(model_output)

    #Create your model
    train_model = keras.models.Model(inputs=model_input, outputs=model_output)

    #Compile your model
    train_model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001),
                        loss='mean_squared_error',
                        metrics=['accuracy'],
                        target_tensors=[outputs])
    return train_model

# Training using `.tfrecords` and `tf.keras.model.fit_generator`
* [Source](https://stackoverflow.com/questions/55243016/training-keras-model-with-fit-generator-and-tfrecords)

In [9]:
def create_dataset_generator(path='',shuffle_buffer_size=None, batch_size=None, compression='GZIP', mode='train'):
    filenames = [file for file in os.listdir(os.path.join(os.getcwd(), path)) if file.endswith('.tfrecord')]
    filenames = [os.path.join(path, file) for file in filenames]
    
    # This works with arrays as well
    dataset = tf.data.TFRecordDataset(filenames, compression_type=compression)
    
    # Maps the parser on every filepath in the array. You can set the number of parallel loaders here
    dataset = dataset.map(parser, num_parallel_calls=8)
    
    # This dataset will go on forever
    dataset = dataset.repeat()
        
    # Set the number of datapoints you want to load and shuffle
    #if(mode=='train'):
    
    dataset = dataset.shuffle(shuffle_buffer_size)
    
    # Set the batchsize
    dataset = dataset.batch(batch_size)
    
    # Create an iterator
    iterator = dataset.make_one_shot_iterator()
    
    # Create your tf representation of the iterator
    image, label = iterator.get_next()

    # Bring your picture back in shape
    image = tf.reshape(image, [batch_size, 3630])
    
    # Create a one hot array for your labels
    label = tf.one_hot(label, 47)
    
    #return image, label
    while True:
        yield image, label

In [10]:
print(create_dataset_generator('40_tfrecords/train',SHUFFLE_BUFFER, BATCH_SIZE))

<generator object create_dataset_generator at 0x7f8cadbe56d0>


In [11]:
#Get your datatensors
image, label = next(create_dataset_generator('40_tfrecords/train',SHUFFLE_BUFFER, BATCH_SIZE))

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [12]:
train_model = smol_model(inputs=image, outputs=label)
train_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(32, 3630)]              0         
_________________________________________________________________
dense (Dense)                (32, 1500)                5446500   
_________________________________________________________________
dense_1 (Dense)              (32, 47)                  70547     
Total params: 5,517,047
Trainable params: 5,517,047
Non-trainable params: 0
_________________________________________________________________


# Start training the model

In [13]:
#Train the model
generator = create_dataset_generator('40_tfrecords/train',SHUFFLE_BUFFER, BATCH_SIZE)
train_model.fit_generator(generator=generator, epochs=EPOCHS, max_queue_size=1000,
                          steps_per_epoch=STEPS_PER_EPOCH)

Epoch 1/3
3525/3524 [==============================] - 16s 5ms/step - loss: 0.0095 - acc: 0.7015
Epoch 2/3
3525/3524 [==============================] - 16s 4ms/step - loss: 0.0050 - acc: 0.8521
Epoch 3/3
3525/3524 [==============================] - 16s 4ms/step - loss: 0.0034 - acc: 0.8997


# Test the model

* Evaluating at small batch sizes (same as training)

In [14]:
#Test the model
BATCH_SIZE = 32
STEPS = 18800/BATCH_SIZE  # number of batches per epoch.
SHUFFLE_BUFFER = BATCH_SIZE**2
generator = create_dataset_generator('40_tfrecords/test', SHUFFLE_BUFFER, BATCH_SIZE,mode='test')
train_model.evaluate_generator(generator=generator, steps=STEPS)

[0.0028191369458928674, 0.9169324]

* Evaluating all of the test data at once.

In [15]:
#Test the model
BATCH_SIZE = 18800
STEPS = 18800/BATCH_SIZE  # number of batches per epoch.
SHUFFLE_BUFFER = BATCH_SIZE**2
generator = create_dataset_generator('40_tfrecords/test', SHUFFLE_BUFFER, BATCH_SIZE,mode='test')
train_model.evaluate_generator(generator=generator, steps=STEPS)

[0.004615416284650564, 0.875]

* There is a small discrepency in the two accuracies, try to find out why is that happening. 

# Restart the network to free up the `GPU` and `RAM`.

In [16]:
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}

# Exercise: Log the RAM usage for the three cases.
* How the RAM usage varies when training the model using an iterators of `.tfrecord` vs direct `NumPy` array vs using a generator of `tfrecords`.
* How does the speed vary?
* Plot various metrics like `training cost vs epochs`, `training accuracy vs epochs` etc. These metrics can be found in the dictionary `history` returned by the model.